# Taller: Modelación del Transporte de Contaminantes en Canales Abiertos
## Notebook 1: Fundamentos y Visualización de Términos

En este notebook exploraremos detalladamente cada uno de los términos de la ecuación de advección-difusión-reacción, visualizando sus efectos por separado y en combinación. Este enfoque nos permitirá desarrollar una intuición física sobre cómo los diferentes procesos afectan el transporte de contaminantes.

In [ ]:
# Importamos las bibliotecas necesarias
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from matplotlib.animation import FuncAnimation
from IPython.display import display, HTML

# Configuración de matplotlib
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## 1. La Ecuación de Advección-Difusión-Reacción

Recordemos que la ecuación que modeliza el transporte de contaminantes en un canal es:

$$\frac{\partial C}{\partial t} + U\frac{\partial C}{\partial x} = D\frac{\partial^2 C}{\partial x^2} - kC$$

Donde:
- $C(x,t)$ es la concentración del contaminante [mg/L]
- $t$ es el tiempo [s]
- $x$ es la posición a lo largo del canal [m]
- $U$ es la velocidad media del flujo [m/s]
- $D$ es el coeficiente de dispersión longitudinal [m²/s]
- $k$ es la constante de degradación [1/s]

Para entender mejor cada término, vamos a reescribir la ecuación agrupando los términos en el lado derecho:

$$\frac{\partial C}{\partial t} = -U\frac{\partial C}{\partial x} + D\frac{\partial^2 C}{\partial x^2} - kC$$

Esta forma nos permite entender más claramente cómo cada término contribuye al cambio de la concentración en el tiempo:

- Término advectivo: $-U\frac{\partial C}{\partial x}$
- Término difusivo: $D\frac{\partial^2 C}{\partial x^2}$
- Término reactivo: $-kC$

A continuación, analizaremos cada término por separado.

## 2. Visualización del Término Advectivo

La advección es el transporte del contaminante debido al movimiento del fluido (agua). Si consideramos solo este término, la ecuación simplificada sería:

$$\frac{\partial C}{\partial t} = -U\frac{\partial C}{\partial x}$$

Esta ecuación describe un transporte puro donde la forma del pulso de contaminante se mantiene, pero su posición cambia a una velocidad $U$. Vamos a implementar una solución numérica simple para visualizarlo:

In [3]:
# Importamos las bibliotecas necesarias
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Configuración de matplotlib
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

def advection_step(C, U, dx, dt):
    """Implementa un paso de tiempo para la ecuación de advección usando un esquema upwind"""
    n = len(C)
    C_new = np.zeros_like(C)
    
    # Esquema upwind para estabilidad
    for i in range(1, n):
        C_new[i] = C[i] - U * (dt/dx) * (C[i] - C[i-1])
    
    # Condición de frontera en x=0
    C_new[0] = C[0]
    
    return C_new

In [ ]:
def simulate_advection():
    """Simula y visualiza el efecto del término advectivo"""
    # Parámetros de la simulación
    L = 100.0        # Longitud del dominio [m]
    nx = 200         # Número de puntos espaciales
    dx = L / nx      # Tamaño del paso espacial
    U = 1.0          # Velocidad [m/s]
    dt = 0.5 * dx / U  # Paso de tiempo que cumple CFL (Courant < 0.5)
    nt = 100         # Número de pasos temporales
    
    # Malla espacial
    x = np.linspace(0, L, nx)
    
    # Condición inicial: pulso gaussiano
    sigma = 5.0
    mu = 20.0
    C_initial = np.exp(-0.5 * ((x - mu) / sigma) ** 2)
    
    # Vector para almacenar resultados cada 10 pasos
    results = [C_initial.copy()]
    times = [0]
    
    # Simulación
    C = C_initial.copy()
    for t in range(1, nt+1):
        C = advection_step(C, U, dx, dt)
        
        if t % 10 == 0 or t == nt:
            results.append(C.copy())
            times.append(t * dt)
    
    # Visualización
    plt.figure(figsize=(12, 6))
    for i, (C_result, time) in enumerate(zip(results, times)):
        plt.plot(x, C_result, label=f't = {time:.1f} s')
    
    plt.xlabel('Distancia (m)')
    plt.ylabel('Concentración')
    plt.title('Advección pura: Transporte sin cambio de forma (U = 1 m/s)')
    plt.legend()
    plt.ylim(0, 1.1)
    plt.grid(True)
    plt.show()
    
    # Animación
    fig, ax = plt.subplots(figsize=(10, 6))
    line, = ax.plot(x, C_initial)
    ax.set_xlabel('Distancia (m)')
    ax.set_ylabel('Concentración')
    ax.set_title('Advección pura: Animación del transporte')
    ax.set_ylim(0, 1.1)
    ax.grid(True)
    
    def update(frame):
        line.set_ydata(results[frame])
        ax.set_title(f'Advección pura: t = {times[frame]:.1f} s')
        return line,
    
    anim = FuncAnimation(fig, update, frames=len(results), interval=200, blit=True)
    plt.close()  # Evita mostrar la figura dos veces
    
    return HTML(anim.to_jshtml())

# Ejecutamos la simulación y visualización
simulate_advection()

### Exploración interactiva del término advectivo

Ahora veamos cómo diferentes velocidades afectan el transporte advectivo:

In [ ]:
# Importamos las bibliotecas necesarias
import numpy as np
import matplotlib.pyplot as plt

def advection_multiple_cases():
    """Visualiza varios casos de advección simultáneamente"""
    # Parámetros
    L = 100.0      # Longitud [m]
    nx = 200       # Puntos espaciales
    x = np.linspace(0, L, nx)
    
    # Condición inicial (t=0)
    sigma = 5.0    # Ancho del pulso
    mu_0 = 20.0    # Posición inicial
    C_0 = np.exp(-0.5 * ((x - mu_0) / sigma) ** 2)
    
    # Velocidades y tiempos a visualizar
    velocities = [0.5, 1.0, 1.5]
    time = 40  # Tiempo fijo
    
    # Visualización
    plt.figure(figsize=(12, 6))
    plt.plot(x, C_0, 'k--', label='Inicial (t=0)')
    
    for velocity in velocities:
        # Solución analítica para advección pura
        mu_t = mu_0 + velocity * time  # Posición en el tiempo t
        C_t = np.exp(-0.5 * ((x - mu_t) / sigma) ** 2)
        plt.plot(x, C_t, label=f'U = {velocity} m/s, t = {time} s')
    
    plt.xlabel('Distancia (m)')
    plt.ylabel('Concentración relativa')
    plt.title(f'Efecto de diferentes velocidades de advección en t = {time} s')
    plt.ylim(0, 1.1)
    plt.xlim(0, L)
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # Ahora mostramos diferentes tiempos para una velocidad fija
    plt.figure(figsize=(12, 6))
    plt.plot(x, C_0, 'k--', label='Inicial (t=0)')
    
    velocity = 1.0  # Velocidad fija
    times = [20, 40, 60]
    
    for time in times:
        # Solución analítica para advección pura
        mu_t = mu_0 + velocity * time  # Posición en el tiempo t
        C_t = np.exp(-0.5 * ((x - mu_t) / sigma) ** 2)
        plt.plot(x, C_t, label=f't = {time} s')
    
    plt.xlabel('Distancia (m)')
    plt.ylabel('Concentración relativa')
    plt.title(f'Efecto de la advección a diferentes tiempos (U = {velocity} m/s)')
    plt.ylim(0, 1.1)
    plt.xlim(0, L)
    plt.legend()
    plt.grid(True)
    plt.show()

# Ejecutar la función
advection_multiple_cases()

### Aspectos clave del término advectivo

- La advección traslada el contaminante sin cambiar su forma
- La velocidad de transporte es igual a la velocidad del flujo $U$
- Después de un tiempo $t$, el contaminante se habrá desplazado una distancia $U \cdot t$
- La masa total del contaminante se conserva (sin pérdidas ni ganancias)
- Las concentraciones máximas permanecen iguales

En términos numéricos, debemos ser cuidadosos con la discretización del término advectivo, ya que puede generar inestabilidades si no se elige adecuadamente el esquema y el paso de tiempo.

## 3. Visualización del Término Difusivo

La difusión (o dispersión en canales) es el esparcimiento del contaminante debido a gradientes de concentración y turbulencia. Si consideramos solo este término, la ecuación simplificada sería:

$$\frac{\partial C}{\partial t} = D\frac{\partial^2 C}{\partial x^2}$$

Esta ecuación describe un proceso de difusión pura, donde el contaminante se esparce desde regiones de alta concentración hacia regiones de baja concentración. Vamos a implementar una solución numérica:

### Exploración interactiva del término difusivo

Veamos cómo diferentes coeficientes de difusión afectan el esparcimiento del contaminante:

In [ ]:
def diffusion_step(C, D, dx, dt):
    """Implementa un paso de tiempo para la ecuación de difusión"""
    n = len(C)
    C_new = np.zeros_like(C)
    
    # Esquema de diferencias centrales para el término difusivo
    for i in range(1, n-1):
        C_new[i] = C[i] + D * (dt/dx**2) * (C[i+1] - 2*C[i] + C[i-1])
    
    # Condiciones de frontera (gradiente cero)
    C_new[0] = C[0] + D * (dt/dx**2) * (C[1] - C[0])  # Aproximación de frontera
    C_new[n-1] = C[n-1] + D * (dt/dx**2) * (C[n-2] - C[n-1])  # Aproximación de frontera
    
    return C_new

def simulate_diffusion():
    """Simula y visualiza el efecto del término difusivo"""
    # Parámetros de la simulación
    L = 100.0        # Longitud del dominio [m]
    nx = 200         # Número de puntos espaciales
    dx = L / nx      # Tamaño del paso espacial
    D = 2.0          # Coeficiente de difusión [m²/s]
    dt = 0.25 * dx**2 / D  # Paso de tiempo que cumple la condición de estabilidad
    nt = 200         # Número de pasos temporales
    
    # Malla espacial
    x = np.linspace(0, L, nx)
    
    # Condición inicial: pulso gaussiano
    sigma = 5.0
    mu = 50.0  # Centrado en el dominio
    C_initial = np.exp(-0.5 * ((x - mu) / sigma) ** 2)
    
    # Vector para almacenar resultados cada 20 pasos
    results = [C_initial.copy()]
    times = [0]
    
    # Simulación
    C = C_initial.copy()
    for t in range(1, nt+1):
        C = diffusion_step(C, D, dx, dt)
        
        if t % 40 == 0 or t == nt:
            results.append(C.copy())
            times.append(t * dt)
    
    # Visualización
    plt.figure(figsize=(12, 6))
    for i, (C_result, time) in enumerate(zip(results, times)):
        plt.plot(x, C_result, label=f't = {time:.1f} s')
    
    plt.xlabel('Distancia (m)')
    plt.ylabel('Concentración')
    plt.title('Difusión pura: Esparcimiento del contaminante (D = 2 m²/s)')
    plt.legend()
    plt.ylim(0, 1.1)
    plt.grid(True)
    plt.show()
    
    # Animación
    fig, ax = plt.subplots(figsize=(10, 6))
    line, = ax.plot(x, C_initial)
    ax.set_xlabel('Distancia (m)')
    ax.set_ylabel('Concentración')
    ax.set_title('Difusión pura: Animación del esparcimiento')
    ax.set_ylim(0, 1.1)
    ax.grid(True)
    
    def update(frame):
        line.set_ydata(results[frame])
        ax.set_title(f'Difusión pura: t = {times[frame]:.1f} s')
        return line,
    
    anim = FuncAnimation(fig, update, frames=len(results), interval=200, blit=True)
    plt.close()
    
    return HTML(anim.to_jshtml())

# Ejecutamos la simulación y visualización
simulate_diffusion()

In [ ]:
# Importamos las bibliotecas necesarias
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

# Configuración de fuente más compatible con caracteres especiales
mpl.rcParams['font.family'] = 'DejaVu Sans'

# Desactivar advertencias específicas (opcional)
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")

def diffusion_multiple_cases():
    """Visualiza múltiples casos de difusión"""
    # Parámetros
    L = 100.0      # Longitud [m]
    nx = 200       # Puntos espaciales
    x = np.linspace(0, L, nx)
    
    # Condición inicial (t=0)
    sigma_0 = 5.0    # Ancho inicial del pulso
    mu = 50.0        # Posición central
    C_0 = np.exp(-0.5 * ((x - mu) / sigma_0) ** 2)
    
    # Difusiones y tiempos a visualizar
    diffusions = [1.0, 2.0, 5.0]
    time = 10  # Tiempo fijo
    
    # Visualización para diferentes coeficientes de difusión
    plt.figure(figsize=(12, 6))
    plt.plot(x, C_0, 'k--', label='Inicial (t=0)')
    
    for diffusion in diffusions:
        # Solución analítica para difusión de un pulso gaussiano
        sigma_t = np.sqrt(sigma_0**2 + 2 * diffusion * time)  # Ensanchamiento
        amplitude = sigma_0 / sigma_t  # Reducción de altura (conservación de masa)
        C_t = amplitude * np.exp(-0.5 * ((x - mu) / sigma_t) ** 2)
        plt.plot(x, C_t, label=f'D = {diffusion} m²/s')
    
    plt.xlabel('Distancia (m)')
    plt.ylabel('Concentración relativa')
    plt.title(f'Efecto de diferentes coeficientes de difusión en t = {time} s')
    plt.ylim(0, 1.1)
    plt.xlim(20, 80)  # Enfocamos en la región central
    plt.legend()
    plt.grid(True)
    
    # Añadimos información sobre el ensanchamiento usando solo caracteres ASCII
    plt.text(27, 0.85, f'Sigma inicial = {sigma_0:.1f} m', fontsize=10)
    for diffusion in diffusions:
        sigma_t = np.sqrt(sigma_0**2 + 2 * diffusion * time)
        plt.text(27, 0.8 - 0.1*diffusions.index(diffusion), 
                f'D = {diffusion} m²/s -> sigma = {sigma_t:.1f} m', 
                fontsize=10)
    
    plt.show()
    
    # Visualización para diferentes tiempos
    plt.figure(figsize=(12, 6))
    plt.plot(x, C_0, 'k--', label='Inicial (t=0)')
    
    diffusion = 2.0  # Difusión fija
    times = [5, 10, 20]
    
    for time in times:
        # Solución analítica para difusión de un pulso gaussiano
        sigma_t = np.sqrt(sigma_0**2 + 2 * diffusion * time)  # Ensanchamiento
        amplitude = sigma_0 / sigma_t  # Reducción de altura (conservación de masa)
        C_t = amplitude * np.exp(-0.5 * ((x - mu) / sigma_t) ** 2)
        plt.plot(x, C_t, label=f't = {time} s')
    
    plt.xlabel('Distancia (m)')
    plt.ylabel('Concentración relativa')
    plt.title(f'Evolución temporal de difusión (D = {diffusion} m²/s)')
    plt.ylim(0, 1.1)
    plt.xlim(20, 80)  # Enfocamos en la región central
    plt.legend()
    plt.grid(True)
    
    # Añadimos información sobre el ensanchamiento
    plt.text(27, 0.85, f'Sigma inicial = {sigma_0:.1f} m', fontsize=10)
    for time in times:
        sigma_t = np.sqrt(sigma_0**2 + 2 * diffusion * time)
        amplitude = sigma_0 / sigma_t
        plt.text(27, 0.8 - 0.1*times.index(time), 
                f't = {time} s -> sigma = {sigma_t:.1f} m (reducción: {(1-amplitude)*100:.1f}%)', 
                fontsize=10)
    
    plt.show()
    
    # Tabla de información
    print("Efecto de la difusión en un pulso gaussiano:")
    print("-" * 60)
    print(f"{'Difusión (m²/s)':<15} | {'Tiempo (s)':<10} | {'Sigma (m)':<10} | {'Reducción (%)':<15}")
    print("-" * 60)
    
    for diffusion in [0.5, 1.0, 2.0, 5.0]:
        for time in [5, 10, 20]:
            sigma_t = np.sqrt(sigma_0**2 + 2 * diffusion * time)
            amplitude = sigma_0 / sigma_t
            print(f"{diffusion:<15} | {time:<10} | {sigma_t:<10.2f} | {(1-amplitude)*100:<15.1f}")

# Ejecutar la función
diffusion_multiple_cases()

### Aspectos clave del término difusivo

- La difusión esparce el contaminante desde regiones de alta a baja concentración
- La tasa de ensanchamiento depende del coeficiente de difusión $D$
- La forma gaussiana se mantiene si la condición inicial es gaussiana
- La masa total del contaminante se conserva
- La concentración máxima disminuye con el tiempo
- Para un pulso gaussiano, el ancho crece según $\sigma(t) = \sqrt{\sigma_0^2 + 2Dt}$

En canales abiertos, además de la difusión molecular, existe la dispersión por cizallamiento debido a las variaciones de velocidad en la sección transversal, lo que aumenta significativamente el coeficiente aparente de difusión/dispersión.

## 4. Visualización del Término Reactivo

El término reactivo representa la degradación (o generación) del contaminante debido a procesos químicos o biológicos. Si consideramos solo este término, la ecuación simplificada sería:

$$\frac{\partial C}{\partial t} = -kC$$

Esta es una ecuación diferencial ordinaria que tiene una solución analítica sencilla: $C(t) = C_0 e^{-kt}$, donde $C_0$ es la concentración inicial. Vamos a visualizar este comportamiento:

In [ ]:
def reaction_step(C, k, dt):
    """Implementa un paso de tiempo para la ecuación de reacción"""
    # La solución exacta para un paso de tiempo es:
    return C * np.exp(-k * dt)

def simulate_reaction():
    """Simula y visualiza el efecto del término reactivo"""
    # Parámetros de la simulación
    L = 100.0        # Longitud del dominio [m]
    nx = 200         # Número de puntos espaciales
    k = 0.05         # Constante de degradación [1/s]
    dt = 1.0         # Paso de tiempo [s]
    nt = 100         # Número de pasos temporales
    
    # Malla espacial
    x = np.linspace(0, L, nx)
    
    # Condición inicial: pulso gaussiano
    sigma = 10.0
    mu = 50.0  # Centrado en el dominio
    C_initial = np.exp(-0.5 * ((x - mu) / sigma) ** 2)
    
    # Vector para almacenar resultados cada 20 pasos
    results = [C_initial.copy()]
    times = [0]
    
    # Simulación
    C = C_initial.copy()
    for t in range(1, nt+1):
        C = reaction_step(C, k, dt)
        
        if t % 20 == 0 or t == nt:
            results.append(C.copy())
            times.append(t * dt)
    
    # Visualización
    plt.figure(figsize=(12, 6))
    for i, (C_result, time) in enumerate(zip(results, times)):
        plt.plot(x, C_result, label=f't = {time:.1f} s')
    
    plt.xlabel('Distancia (m)')
    plt.ylabel('Concentración')
    plt.title('Reacción pura: Degradación del contaminante (k = 0.05 1/s)')
    plt.legend()
    plt.ylim(0, 1.1)
    plt.grid(True)
    plt.show()
    
    # Animación
    fig, ax = plt.subplots(figsize=(10, 6))
    line, = ax.plot(x, C_initial)
    ax.set_xlabel('Distancia (m)')
    ax.set_ylabel('Concentración')
    ax.set_title('Reacción pura: Animación de la degradación')
    ax.set_ylim(0, 1.1)
    ax.grid(True)
    
    def update(frame):
        line.set_ydata(results[frame])
        ax.set_title(f'Reacción pura: t = {times[frame]:.1f} s')
        return line,
    
    anim = FuncAnimation(fig, update, frames=len(results), interval=200, blit=True)
    plt.close()
    
    return HTML(anim.to_jshtml())

# Ejecutamos la simulación y visualización
simulate_reaction()

### Exploración interactiva del término reactivo

Veamos cómo diferentes constantes de degradación afectan la concentración del contaminante:

In [ ]:
# Importamos las bibliotecas necesarias
import numpy as np
import matplotlib.pyplot as plt

def reaction_multiple_cases():
    """Visualiza múltiples casos de reacción"""
    # Parámetros
    L = 100.0      # Longitud [m]
    nx = 200       # Puntos espaciales
    x = np.linspace(0, L, nx)
    
    # Condición inicial (t=0)
    sigma = 10.0     # Ancho del pulso
    mu = 50.0        # Posición central
    C_0 = np.exp(-0.5 * ((x - mu) / sigma) ** 2)
    
    # Tasas de degradación y tiempos a visualizar
    reaction_rates = [0.01, 0.05, 0.1]
    fixed_time = 50  # Tiempo fijo para comparar tasas
    
    # Figura 1: Diferentes tasas de degradación a tiempo fijo
    plt.figure(figsize=(12, 10))
    
    # Gráfico espacial (superior)
    plt.subplot(2, 1, 1)
    plt.plot(x, C_0, 'k--', label='Inicial (t=0)')
    
    for k in reaction_rates:
        C_t = C_0 * np.exp(-k * fixed_time)
        plt.plot(x, C_t, label=f'k = {k:.3f} 1/s')
    
    plt.xlabel('Distancia (m)')
    plt.ylabel('Concentración')
    plt.title(f'Degradación del contaminante a t = {fixed_time} s')
    plt.ylim(0, 1.1)
    plt.xlim(20, 80)  # Enfocamos en la región central
    plt.legend()
    plt.grid(True)
    
    # Gráfico temporal (inferior)
    plt.subplot(2, 1, 2)
    time_array = np.linspace(0, 100, 200)
    
    for k in reaction_rates:
        C_max_t = np.exp(-k * time_array)
        plt.plot(time_array, C_max_t, label=f'k = {k:.3f} 1/s')
        # Marcar tiempo de vida media
        half_life = np.log(2) / k
        plt.axvline(x=half_life, linestyle='--', alpha=0.5)
        plt.text(half_life + 2, 0.5, f't½ = {half_life:.1f} s', fontsize=9)
    
    plt.xlabel('Tiempo (s)')
    plt.ylabel('Concentración relativa máxima')
    plt.title('Evolución temporal para diferentes tasas de degradación')
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Figura 2: Una tasa de degradación a diferentes tiempos
    plt.figure(figsize=(12, 6))
    
    k = 0.05  # Tasa de degradación fija
    times = [0, 10, 25, 50, 100]
    
    for time in times:
        C_t = C_0 * np.exp(-k * time)
        plt.plot(x, C_t, label=f't = {time} s')
    
    plt.xlabel('Distancia (m)')
    plt.ylabel('Concentración')
    plt.title(f'Degradación del contaminante (k = {k:.3f} 1/s)')
    plt.ylim(0, 1.1)
    plt.xlim(20, 80)
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # Tabla informativa
    print("Efecto de la reacción en un contaminante:")
    print("-" * 70)
    print(f"{'Tasa (1/s)':<12} | {'Vida media (s)':<15} | {'Tiempo (s)':<12} | {'Fracción restante (%)':<20}")
    print("-" * 70)
    
    for k in [0.01, 0.02, 0.05, 0.1]:
        half_life = np.log(2) / k
        for time in [10, 50, 100]:
            fraction = np.exp(-k * time) * 100
            print(f"{k:<12.3f} | {half_life:<15.1f} | {time:<12} | {fraction:<20.2f}")

# Ejecutar la función
reaction_multiple_cases()

### Aspectos clave del término reactivo

- La reacción disminuye (o aumenta, si $k < 0$) la concentración de contaminante uniformemente
- La concentración decae exponencialmente: $C(t) = C_0 e^{-kt}$
- El tiempo de vida media del contaminante es $t_{1/2} = \ln(2)/k$
- La forma espacial del perfil de concentración se mantiene
- La masa total del contaminante disminuye con el tiempo
- El proceso es independiente de la posición (cada punto reacciona localmente)

En el contexto de contaminantes en canales, el término reactivo puede representar degradación biológica, oxidación química, fotólisis, adsorción a sedimentos, entre otros procesos.

## 5. Combinación de Términos

Ahora que hemos analizado cada término por separado, veamos cómo se comporta la ecuación completa que combina advección, difusión y reacción simultáneamente. Implementaremos un esquema numérico que integra los tres procesos:

In [ ]:
def adr_step(C, U, D, k, dx, dt):
    """Implementa un paso de tiempo para la ecuación de advección-difusión-reacción"""
    n = len(C)
    C_new = np.zeros_like(C)
    
    # Aplicamos todos los términos juntos para los puntos interiores
    for i in range(1, n-1):
        # Término advectivo (upwind)
        advection = -U * (dt/dx) * (C[i] - C[i-1])
        
        # Término difusivo (diferencias centrales)
        diffusion = D * (dt/dx**2) * (C[i+1] - 2*C[i] + C[i-1])
        
        # Término reactivo
        reaction = -k * dt * C[i]
        
        # Combinamos los términos
        C_new[i] = C[i] + advection + diffusion + reaction
    
    # Condiciones de frontera simplificadas
    C_new[0] = C[0]  # Concentración fija en x=0
    C_new[n-1] = C[n-2]  # Gradiente cero en x=L
    
    return C_new

def simulate_adr():
    """Simula y visualiza el efecto combinado de advección, difusión y reacción"""
    # Parámetros de la simulación
    L = 100.0        # Longitud del dominio [m]
    nx = 200         # Número de puntos espaciales
    dx = L / nx      # Tamaño del paso espacial
    U = 1.0          # Velocidad [m/s]
    D = 2.0          # Coeficiente de difusión [m²/s]
    k = 0.02         # Constante de degradación [1/s]
    
    # Paso de tiempo que cumple la condición de estabilidad
    dt = 0.25 * min(dx/U, dx**2/(2*D))
    nt = 500         # Número de pasos temporales
    
    # Malla espacial
    x = np.linspace(0, L, nx)
    
    # Condición inicial: pulso gaussiano
    sigma = 5.0
    mu = 20.0  # Posición inicial
    C_initial = np.exp(-0.5 * ((x - mu) / sigma) ** 2)
    
    # Vector para almacenar resultados cada cierto número de pasos
    results = [C_initial.copy()]
    times = [0]
    
    # Simulación
    C = C_initial.copy()
    for t in range(1, nt+1):
        C = adr_step(C, U, D, k, dx, dt)
        
        if t % 100 == 0 or t == nt:
            results.append(C.copy())
            times.append(t * dt)
    
    # Visualización
    plt.figure(figsize=(12, 6))
    for i, (C_result, time) in enumerate(zip(results, times)):
        plt.plot(x, C_result, label=f't = {time:.1f} s')
    
    plt.xlabel('Distancia (m)')
    plt.ylabel('Concentración')
    plt.title(f'Advección-Difusión-Reacción: U={U} m/s, D={D} m²/s, k={k} 1/s')
    plt.legend()
    plt.ylim(0, 1.1)
    plt.grid(True)
    plt.show()
    
    # Animación
    fig, ax = plt.subplots(figsize=(10, 6))
    line, = ax.plot(x, C_initial)
    ax.set_xlabel('Distancia (m)')
    ax.set_ylabel('Concentración')
    ax.set_title('Animación del transporte completo')
    ax.set_ylim(0, 1.1)
    ax.grid(True)
    
    def update(frame):
        line.set_ydata(results[frame])
        ax.set_title(f'Transporte ADR: t = {times[frame]:.1f} s')
        return line,
    
    anim = FuncAnimation(fig, update, frames=len(results), interval=200, blit=True)
    plt.close()
    
    return HTML(anim.to_jshtml())

# Ejecutamos la simulación y visualización
simulate_adr()

### Exploración interactiva de la ecuación completa

Ahora exploremos el comportamiento de la ecuación completa con diferentes combinaciones de parámetros:

### Aspectos clave de la ecuación completa

Cuando los tres procesos (advección, difusión y reacción) actúan simultáneamente:

1. **Advección**: Traslada el pulso a lo largo del canal sin cambiar su forma
2. **Difusión**: Ensancha el pulso y reduce su altura, manteniendo la masa total
3. **Reacción**: Reduce la concentración uniformemente, disminuyendo la masa total

El comportamiento dominante dependerá de la magnitud relativa de los parámetros, que se pueden expresar mediante números adimensionales:

- **Número de Péclet** ($Pe = UL/D$): Ratio entre advección y difusión
  - $Pe \gg 1$: Domina la advección
  - $Pe \ll 1$: Domina la difusión

- **Número de Damköhler** ($Da = kL/U$): Ratio entre reacción y advección
  - $Da \gg 1$: Domina la reacción
  - $Da \ll 1$: Domina la advección

En el próximo notebook, profundizaremos en estos números adimensionales y su importancia para caracterizar el comportamiento del sistema.

## 6. Ejercicios Prácticos

A continuación, se proponen algunos ejercicios para reforzar la comprensión de los conceptos estudiados:

### Ejercicio 1: Efectos relativos de los términos

Modifica la función `adr_interactive` para comparar casos donde:

1. Solo actúe la advección (D=0, k=0)
2. Solo actúe la difusión (U=0, k=0)
3. Solo actúe la reacción (U=0, D=0)
4. Actúen todos los términos juntos

Muestra los cuatro casos en una misma gráfica para facilitar la comparación.

In [ ]:
# Tu solución al Ejercicio 1 aquí
# ...


### Ejercicio 2: Conservación de masa

Implementa una función que calcule y visualice la evolución de la masa total del contaminante en función del tiempo para diferentes valores del coeficiente de reacción k.

La masa total se puede calcular aproximadamente como: $M(t) = \Delta x \sum_i C(x_i, t)$

In [ ]:
# Tu solución al Ejercicio 2 aquí
# ...


### Ejercicio 3: Solución analítica vs. numérica

Compara la solución numérica obtenida con la función `adr_step` con la aproximación analítica utilizada en `adr_interactive` para un conjunto específico de parámetros. Calcula y visualiza el error absoluto entre ambas soluciones.

In [ ]:
# Tu solución al Ejercicio 3 aquí
# ...


## 7. Conclusiones

En este notebook, hemos explorado detalladamente los efectos individuales y combinados de los términos de la ecuación de advección-difusión-reacción:

1. El **término advectivo** traslada el contaminante sin cambiar su forma ni su masa total
2. El **término difusivo** ensancha el pulso y reduce su altura, conservando la masa total
3. El **término reactivo** reduce la concentración exponencialmente, disminuyendo la masa total

Hemos visualizado estos efectos tanto por separado como en combinación, y hemos implementado un esquema numérico básico para la resolución de la ecuación completa.

En el próximo notebook, abordaremos la adimensionalización de esta ecuación, lo que nos permitirá identificar los parámetros adimensionales que gobiernan el comportamiento del sistema y simplificar su análisis.